# Imports and initialization

In [ ]:
from __future__ import print_function

from github_settings import (ry_username, ry_password,
                             username, password,
                             token, 
                             GITENBERG_GITHUB_TOKEN,
                             GITENBERG_TRAVIS_ACCESS_TOKEN,
                             RDHYEE_GITHUB_TOKEN,
                             RDHYEE_TRAVIS_ACCESS_TOKEN, 
                             RDHYEE_TRAVIS_PROFILE_TOKEN)


from itertools import (islice, izip)
import requests


from second_folio import (apply_to_repos, all_repos)

from gitenberg_utils import (GitenbergJob,
                             GitenbergTravisJob,
                             ForkBuildRepo,
                             BuildRepo,
                             BuildRepo2,
                             MetadataWrite,
                             RepoNameFixer,
                             repo_md)


import pandas as pd
from pandas import (DataFrame, Series)

In [ ]:
# loop through all the repos

repos = all_repos


# Figure out the various tokens

There are at least three types of tokens in use in `travis-ci` ([The Travis CI Blog: Token, Token, Token](https://blog.travis-ci.com/2013-01-28-token-token-token/)):

* GitHub token
* access token
* Travis token



In [ ]:
# try using an access token

from travispy import TravisPy
#travis_client = TravisPy(RDHYEE_TRAVIS_ACCESS_TOKEN)
travis_client = TravisPy(GITENBERG_TRAVIS_ACCESS_TOKEN)

travis_repo = travis_client.repo("GITenberg/Adventures-of-Huckleberry-Finn_76")

In [ ]:
travis_repo.enable()

In [ ]:
# using GitHub auth and read off the corresponding access token

from travispy import TravisPy
#t = TravisPy.github_auth(RDHYEE_GITHUB_TOKEN)
t = TravisPy.github_auth(GITENBERG_GITHUB_TOKEN)


In [ ]:
session = t._session
(session.uri,  
 session.headers.get('Authorization').split()[-1] == GITENBERG_TRAVIS_ACCESS_TOKEN)

# everything else

In [ ]:
# repo = "Anne-of-the-Island_51"
# repo = "Adventures-of-Huckleberry-Finn_76"
repo = "Chaucer-s-Works-Volume-4--of-7----The-Canterbury-Tales_22120"

bj = BuildRepo(username=ry_username, password=ry_password, repo_name=repo,
              repo_owner='rdhyee', 
              update_travis_commit_msg='try again: encoding problem?',
              tag_commit_message='try again: encoding problem?',
              access_token=RDHYEE_TRAVIS_ACCESS_TOKEN)

In [ ]:
bj.run()

# fork and build

In [ ]:
# fork and build repo

repo = repos[-1]

bj2 = ForkBuildRepo(username=ry_username, password=ry_password, repo_name=repo,
              repo_owner='rdhyee', 
              update_travis_commit_msg='update travis',
              tag_commit_message='update travis',
              access_token=RDHYEE_TRAVIS_ACCESS_TOKEN)



In [ ]:
bj2.fork_repo()

In [ ]:
from collections import OrderedDict
from itertools import islice

results = OrderedDict()

repos = all_repos
repos.reverse()

repos_iter = iter(repos)

In [ ]:
def build_repos(repos, n=None):
    for (i, repo) in enumerate(islice(repos, n)):
        try:
            bj = BuildRepo(username=ry_username, password=ry_password, repo_name=repo,
                  repo_owner='rdhyee', 
                  update_travis_commit_msg='build using gitenberg.travis',
                  tag_commit_message='build using gitenberg.travis',
                  access_token=RDHYEE_TRAVIS_ACCESS_TOKEN)
            results[repo] = (bj, bj.run())
        except Exception, e:
            results[repo] = e
        
        print ("\r{}: {}".format(i, results[repo]), end="")
    

In [ ]:
results.values()[-1][0].repo_name

In [ ]:
build_repos(repos_iter, 32)

In [ ]:
len(results)

In [ ]:
[result[0].status().get('ebooks_in_release_count') for result in results.values()[-5:]]

In [ ]:
results.values()[-1][0].status()

# if I want to delete existing repo tokens...

can use

```
auths = gh.iter_authorizations()
[(auth.name, auth.created_at) for auth in auths]
```

# try rebuilding errored builds

In [ ]:
from collections import OrderedDict
from itertools import islice

results = OrderedDict()

repos = all_repos
repos.reverse()

repos_iter = iter(repos)

def rebuild_repos(repos, n=None):
    for (i, repo) in enumerate(islice(repos, n)):
        try:
            bj = BuildRepo(username=ry_username, password=ry_password, repo_name=repo,
              repo_owner='rdhyee', 
              update_travis_commit_msg='build using gitenberg.travis (retry)',
              tag_commit_message='build using gitenberg.travis (retry)',
              access_token=RDHYEE_TRAVIS_ACCESS_TOKEN)
            # reset token
            results[repo] = (bj, (load_repo_token=False))
        except Exception, e:
            results[repo] = e
        
        print ("\r{}: {}".format(i, results[repo]), end="")

In [ ]:
rebuild_repos(repos_iter,None)

In [ ]:
len(results)

In [ ]:
repos_failed = [result[0].repo_name for result in results.values() if result[1] == 'failed']
repos_failed

# fix repo names

In [ ]:
REPOS_FAILED = ['The-Pilgrim-s-Progress-from-this-world-to-that-which-is-to-come--13-Delivered-under-the-similit__131',
'The-Art-of-War_132',
'The-Thousand-and-One-Nights-Vol.-I.Commonly-Called-the-Arabian-Nights-Entertainments_34206',
'On-the-Origin-of-Species-By-Means-of-Natural-Selection--13-Or-the-Preservation-of-Favoured-Rac__1228',
'Daddy-Long-Legs_157',
'Heidi-Gift-Edition-_20781',
'The-Trial_7849',
'Chaucer-s-Works-Volume-4--of-7----The-Canterbury-Tales_22120',
'Persuasion_105',
'The-Works-of-Edgar-Allan-Poe-The-Raven-EditionTable-Of-Contents-And-Index-Of-The-Five-Volumes_25525']

In [ ]:
def fixname_repos(repos, n=None):
    for (i, repo) in enumerate(islice(repos, n)):
        try:
            bj = RepoNameFixer(username=ry_username, password=ry_password, repo_name=repo,
              repo_owner='rdhyee', 
              update_travis_commit_msg='build using gitenberg.travis (retry)',
              tag_commit_message='build using gitenberg.travis (retry)',
              access_token=RDHYEE_TRAVIS_ACCESS_TOKEN)
            results[repo] = (bj, bj.run())
        except Exception, e:
            results[repo] = e
        
        print ("\r{}: {}".format(i, results[repo]), end="")

# status of the Second Folio forks (152 repos)? 


In [ ]:
from second_folio import (all_repos)

all_repos[:5]

In [ ]:
def status_for_repo(repo_name):
    rs = GitenbergJob(username=username, password=password, repo_name=repo_name,
              repo_owner='rdhyee', 
              update_travis_commit_msg='check status',
              tag_commit_message='check status')
    return rs.status()

results_iter = apply_to_repos(status_for_repo, repos=repos)

In [ ]:
results = []

for (i,result) in enumerate(results_iter):
    
    results.append(result)
    if not isinstance(result, Exception):
        print ("\r{}: {}".format(i, result['repo_name']), end="")
    else:
        print ("\r{}: {}".format(i, str(result)), end="")

In [ ]:
[(i, result) for (i, result) in enumerate(results) if isinstance(result, Exception)]

In [ ]:
[result.get('repo_name') for result in results if result.get('ebooks_in_release_count') != 3]

In [ ]:
results[0]

# forks still require rebuilding -- including name change

In [ ]:
TO_REBUILD = ['Dubliners_2814',
 'Moby-Dick--Or-The-Whale_2701',
 'The-Brothers-Karamazov_28054',
 'Frankenstein_84',
 'The-Works-of-Edgar-Allan-Poe-The-Raven-EditionTable-Of-Contents-And-Index-Of-The-Five-Volumes_25525',
 'A-Little-Princess--13-Being-the-whole-story-of-Sara-Crewe-now-told-for-the-first-time_146',
 'The-Invisible-Man--A-Grotesque-Romance_5230',
 'Persuasion_105',
 'Tales-of-the-Jazz-Age_6695',
 'The-Last-of-the-Mohicans--A-Narrative-of-1757_27681',
 'Around-the-World-in-80-Days_103',
 'The-Trial_7849',
 'The-Posthumous-Papers-of-the-Pickwick-Club-v-1of-2_47534',
 'Cyrano-De-Bergerac_1254',
 'Daddy-Long-Legs_157',
 'Aesop-s-Fables--a-new-translation_11339',
 'The-Art-of-War_132']

In [ ]:
repo = "The-Posthumous-Papers-of-the-Pickwick-Club-v-1of-2_47534"
# repo = "Adventures-of-Huckleberry-Finn_76"
# repo = "Chaucer-s-Works-Volume-4--of-7----The-Canterbury-Tales_22120"


bj = BuildRepo(username=ry_username, password=ry_password, repo_name=repo,
              repo_owner='rdhyee', 
              update_travis_commit_msg='try again after fixing _version',
              tag_commit_message='try again after fixing _version',
              access_token=RDHYEE_TRAVIS_ACCESS_TOKEN)

In [ ]:
bj.fix_repo_name()

In [ ]:
bj.delete_repo_token()

In [ ]:
bj.run()

In [ ]:
bj.status()

In [ ]:
repos_iter = iter(TO_REBUILD[4:])

In [ ]:
from collections import OrderedDict
results = OrderedDict()

def build_repos(repos, n=None):
    for (i, repo) in enumerate(islice(repos, n)):
        try:
            bj = BuildRepo(username=ry_username, password=ry_password, repo_name=repo,
                  repo_owner='rdhyee', 
                  update_travis_commit_msg='build repo',
                  tag_commit_message='build repo',
                  access_token=RDHYEE_TRAVIS_ACCESS_TOKEN)
            token_delete_result = bj.delete_repo_token()
            results[repo] = (bj, bj.run(load_repo_token=False), token_delete_result)
        except Exception, e:
            results[repo] = e
        
        print ("\r{}: {}".format(i, results[repo]), end="")

In [ ]:
build_repos(repos_iter,None)

In [ ]:
[result[0].repo_name for result in results.values() if result[1] is not None]

In [ ]:
results